# OaaS SDK Tutorial (Runnable)

This notebook demonstrates the simplified OaaS SDK using mock mode so it runs locally without external services. We use top-level `await` in code cells for async calls.


In [1]:
# Core imports
from oaas_sdk2_py import oaas, OaasObject, OaasConfig
from pydantic import BaseModel
from typing import Dict, List, Any

print("OaaS tutorial environment ready: SDK imported.")


OaaS tutorial environment ready: SDK imported.


In [2]:
# Configure OaaS for local/mock mode (no external services)
config = OaasConfig(
    async_mode=True,
    mock_mode=True,
    oprc_partition_default=0,
    auto_commit=True,
)
oaas.configure(config)

# Quick health check
health = oaas.health_check()
print("Health:", health)


Health: {'healthy': True, 'issues': [], 'warnings': ['Global Oparaca instance not initialized', 'AutoSessionManager not initialized', 'No services registered'], 'info': []}


## Define a Service

We define a `Counter` service with persistent fields. Typed attributes automatically persist via the SDK's state descriptor system.


In [ ]:
# Request model
class IncrementRequest(BaseModel):
    amount: int = 1

# Define service
@oaas.service("Counter", package="example")
class Counter(OaasObject):
    count: int = 0
    history: List[str] = []

    @oaas.method()
    async def increment(self, req: IncrementRequest) -> int:
        self.count += req.amount
        self.history.append(f"Incremented by {req.amount}")
        return self.count

    @oaas.method()
    async def get_value(self) -> int:
        return self.count

    @oaas.method()
    async def get_history(self) -> List[str]:
        return self.history

    @oaas.method()
    async def reset(self) -> bool:
        self.count = 0
        self.history.clear()
        return True


2025-08-27 07:21:03,291 - oaas_sdk - INFO - Service Counter successfully registered


## Create a Local Object

For testing, we create a local instance of the `Counter` service. This avoids any external server or agent dependencies.


In [ ]:
# Create a object for testing
counter = Counter.create()
print("Created Counter")
print("object_id:", getattr(counter, "object_id", None))


Created Counter (local)
object_id: 748442011473351867


## Run Method Calls

Now we'll invoke a few methods using `await` and print the results and state. The SDK handles serialization automatically.


In [6]:
# Invoke methods with await
val = await counter.increment(IncrementRequest(amount=5))
print("increment ->", val)

cur = await counter.get_value()
print("get_value ->", cur)

hist = await counter.get_history()
print("get_history ->", hist)

ok = await counter.reset()
print("reset ->", ok)

cur2 = await counter.get_value()
print("after reset get_value ->", cur2)


increment -> 5
get_value -> 5
get_history -> ['Incremented by 5']
reset -> True
after reset get_value -> 0
